### Import

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os

### Checkpoint function

In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

### Data load & pre-processing function

In [3]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

### Performance function

In [4]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

### Model function

In [5]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

### Create model

In [6]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(2):
            self.model.add(dense(256, weight_init))
            self.model.add(relu())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):

        x = self.model(x)

        return x

In [7]:
def create_model_function(label_dim) :
    # weight 결정시 glorot_uniform 사용 (xavier)
    weight_init = tf.keras.initializers.glorot_uniform()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(2) :
        model.add(dense(256, weight_init))
        model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model

### Define data & hyper-parameter

In [8]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

### Define model & optimizer & writer

In [9]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_xavier'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

### Restore checkpoint & start train or test phase

In [10]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):                
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)

                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 1.4013, train_loss: 2.08147812, train_accuracy: 0.3672, test_Accuracy: 0.2380
Epoch: [ 0] [    1/  468] time: 1.7218, train_loss: 1.92245567, train_accuracy: 0.6172, test_Accuracy: 0.5049
Epoch: [ 0] [    2/  468] time: 2.0326, train_loss: 1.85806108, train_accuracy: 0.6328, test_Accuracy: 0.5454
Epoch: [ 0] [    3/  468] time: 2.4544, train_loss: 1.73114896, train_accuracy: 0.6250, test_Accuracy: 0.5632
Epoch: [ 0] [    4/  468] time: 2.7816, train_loss: 1.56156254, train_accuracy: 0.6328, test_Accuracy: 0.5947
Epoch: [ 0] [    5/  468] time: 3.0927, train_loss: 1.50348043, train_accuracy: 0.6406, test_Accuracy: 0.6325
Epoch: [ 0] [    6/  468] time: 3.4039, train_loss: 1.41716695, train_accuracy: 0.6719, test_Accuracy: 0.6929
Epoch: [ 0] [    7/  468] time: 3.7141, train_loss: 1.22120380, train_accuracy: 0.7344, test_Accuracy: 0.7254
Epoch: [ 0] [    8/  468] time: 4.0322,

Epoch: [ 0] [   74/  468] time: 23.7216, train_loss: 0.21538503, train_accuracy: 0.9453, test_Accuracy: 0.9072
Epoch: [ 0] [   75/  468] time: 24.0547, train_loss: 0.33651757, train_accuracy: 0.8750, test_Accuracy: 0.9130
Epoch: [ 0] [   76/  468] time: 24.3813, train_loss: 0.25129989, train_accuracy: 0.9297, test_Accuracy: 0.9145
Epoch: [ 0] [   77/  468] time: 24.6680, train_loss: 0.26509783, train_accuracy: 0.9297, test_Accuracy: 0.9117
Epoch: [ 0] [   78/  468] time: 24.9573, train_loss: 0.23847966, train_accuracy: 0.9375, test_Accuracy: 0.9068
Epoch: [ 0] [   79/  468] time: 25.2575, train_loss: 0.29750913, train_accuracy: 0.9219, test_Accuracy: 0.9025
Epoch: [ 0] [   80/  468] time: 25.5461, train_loss: 0.22661026, train_accuracy: 0.9062, test_Accuracy: 0.9048
Epoch: [ 0] [   81/  468] time: 25.8323, train_loss: 0.21112017, train_accuracy: 0.9375, test_Accuracy: 0.9095
Epoch: [ 0] [   82/  468] time: 26.1694, train_loss: 0.25704366, train_accuracy: 0.9219, test_Accuracy: 0.9147
E

Epoch: [ 0] [  148/  468] time: 46.3066, train_loss: 0.11043516, train_accuracy: 0.9688, test_Accuracy: 0.9391
Epoch: [ 0] [  149/  468] time: 46.6048, train_loss: 0.14170301, train_accuracy: 0.9453, test_Accuracy: 0.9389
Epoch: [ 0] [  150/  468] time: 46.9232, train_loss: 0.16868772, train_accuracy: 0.9531, test_Accuracy: 0.9387
Epoch: [ 0] [  151/  468] time: 47.2600, train_loss: 0.31384423, train_accuracy: 0.9141, test_Accuracy: 0.9380
Epoch: [ 0] [  152/  468] time: 47.5692, train_loss: 0.16708709, train_accuracy: 0.9531, test_Accuracy: 0.9383
Epoch: [ 0] [  153/  468] time: 47.8864, train_loss: 0.16081607, train_accuracy: 0.9609, test_Accuracy: 0.9382
Epoch: [ 0] [  154/  468] time: 48.2065, train_loss: 0.11262933, train_accuracy: 0.9844, test_Accuracy: 0.9389
Epoch: [ 0] [  155/  468] time: 48.5217, train_loss: 0.36318898, train_accuracy: 0.8984, test_Accuracy: 0.9381
Epoch: [ 0] [  156/  468] time: 48.7499, train_loss: 0.33319148, train_accuracy: 0.9297, test_Accuracy: 0.9389
E

Epoch: [ 0] [  222/  468] time: 68.6455, train_loss: 0.23743092, train_accuracy: 0.9375, test_Accuracy: 0.9499
Epoch: [ 0] [  223/  468] time: 68.9766, train_loss: 0.11516812, train_accuracy: 0.9609, test_Accuracy: 0.9497
Epoch: [ 0] [  224/  468] time: 69.2379, train_loss: 0.27230722, train_accuracy: 0.9297, test_Accuracy: 0.9492
Epoch: [ 0] [  225/  468] time: 69.4623, train_loss: 0.25257352, train_accuracy: 0.9062, test_Accuracy: 0.9484
Epoch: [ 0] [  226/  468] time: 69.7865, train_loss: 0.21598500, train_accuracy: 0.9453, test_Accuracy: 0.9478
Epoch: [ 0] [  227/  468] time: 70.1175, train_loss: 0.07979354, train_accuracy: 0.9844, test_Accuracy: 0.9463
Epoch: [ 0] [  228/  468] time: 70.4347, train_loss: 0.12374602, train_accuracy: 0.9688, test_Accuracy: 0.9436
Epoch: [ 0] [  229/  468] time: 70.7708, train_loss: 0.17322859, train_accuracy: 0.9375, test_Accuracy: 0.9437
Epoch: [ 0] [  230/  468] time: 71.0979, train_loss: 0.21057135, train_accuracy: 0.9375, test_Accuracy: 0.9454
E

Epoch: [ 0] [  296/  468] time: 91.0855, train_loss: 0.16625798, train_accuracy: 0.9297, test_Accuracy: 0.9495
Epoch: [ 0] [  297/  468] time: 91.4495, train_loss: 0.14597535, train_accuracy: 0.9375, test_Accuracy: 0.9506
Epoch: [ 0] [  298/  468] time: 91.7656, train_loss: 0.09507933, train_accuracy: 0.9688, test_Accuracy: 0.9512
Epoch: [ 0] [  299/  468] time: 92.1188, train_loss: 0.20772807, train_accuracy: 0.9453, test_Accuracy: 0.9504
Epoch: [ 0] [  300/  468] time: 92.4638, train_loss: 0.14801209, train_accuracy: 0.9375, test_Accuracy: 0.9487
Epoch: [ 0] [  301/  468] time: 92.7919, train_loss: 0.07532057, train_accuracy: 0.9844, test_Accuracy: 0.9459
Epoch: [ 0] [  302/  468] time: 93.0003, train_loss: 0.18614617, train_accuracy: 0.9375, test_Accuracy: 0.9470
Epoch: [ 0] [  303/  468] time: 93.2487, train_loss: 0.13806157, train_accuracy: 0.9453, test_Accuracy: 0.9488
Epoch: [ 0] [  304/  468] time: 93.5608, train_loss: 0.15869029, train_accuracy: 0.9531, test_Accuracy: 0.9499
E

Epoch: [ 0] [  370/  468] time: 114.3703, train_loss: 0.17974511, train_accuracy: 0.9609, test_Accuracy: 0.9578
Epoch: [ 0] [  371/  468] time: 114.6510, train_loss: 0.19028896, train_accuracy: 0.9297, test_Accuracy: 0.9589
Epoch: [ 0] [  372/  468] time: 114.9636, train_loss: 0.12311988, train_accuracy: 0.9609, test_Accuracy: 0.9598
Epoch: [ 0] [  373/  468] time: 115.3178, train_loss: 0.16777733, train_accuracy: 0.9453, test_Accuracy: 0.9611
Epoch: [ 0] [  374/  468] time: 115.6219, train_loss: 0.20681569, train_accuracy: 0.9609, test_Accuracy: 0.9599
Epoch: [ 0] [  375/  468] time: 115.9231, train_loss: 0.12101158, train_accuracy: 0.9609, test_Accuracy: 0.9599
Epoch: [ 0] [  376/  468] time: 116.2442, train_loss: 0.10779321, train_accuracy: 0.9688, test_Accuracy: 0.9603
Epoch: [ 0] [  377/  468] time: 116.5743, train_loss: 0.13254200, train_accuracy: 0.9766, test_Accuracy: 0.9600
Epoch: [ 0] [  378/  468] time: 116.8745, train_loss: 0.17085296, train_accuracy: 0.9531, test_Accuracy:

Epoch: [ 0] [  444/  468] time: 137.0805, train_loss: 0.08033387, train_accuracy: 0.9766, test_Accuracy: 0.9633
Epoch: [ 0] [  445/  468] time: 137.4316, train_loss: 0.08771048, train_accuracy: 0.9844, test_Accuracy: 0.9629
Epoch: [ 0] [  446/  468] time: 137.6561, train_loss: 0.19285598, train_accuracy: 0.9609, test_Accuracy: 0.9626
Epoch: [ 0] [  447/  468] time: 138.0030, train_loss: 0.09607544, train_accuracy: 0.9453, test_Accuracy: 0.9632
Epoch: [ 0] [  448/  468] time: 138.2942, train_loss: 0.10211195, train_accuracy: 0.9766, test_Accuracy: 0.9625
Epoch: [ 0] [  449/  468] time: 138.5675, train_loss: 0.10295539, train_accuracy: 0.9766, test_Accuracy: 0.9634
Epoch: [ 0] [  450/  468] time: 138.8827, train_loss: 0.09666504, train_accuracy: 0.9688, test_Accuracy: 0.9624
Epoch: [ 0] [  451/  468] time: 139.2218, train_loss: 0.10346100, train_accuracy: 0.9609, test_Accuracy: 0.9630
Epoch: [ 0] [  452/  468] time: 139.4960, train_loss: 0.11555018, train_accuracy: 0.9609, test_Accuracy:

### Accuracy : 96.55%